In [1]:
'''
!pip uninstall tensorflow
%tensorflow_version 1.x
!pip uninstall tensorflow
!pip install tensorflow-gpu==1.14.0
'''

'\n!pip uninstall tensorflow\n%tensorflow_version 1.x\n!pip uninstall tensorflow\n!pip install tensorflow-gpu==1.14.0\n'

In [2]:
!wget http://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz
!mkdir -p ~/data/flowers
!tar -xzf 102flowers.tgz -C ~/data/flowers/
!rm 102flowers.tgz
!wget http://www.robots.ox.ac.uk/~vgg/data/flowers/102/setid.mat
!wget http://www.robots.ox.ac.uk/~vgg/data/flowers/102/imagelabels.mat
!mv setid.mat ~/data/flowers/setid.mat
!mv imagelabels.mat ~/data/flowers/imagelabels.mat

--2020-06-05 17:45:07--  http://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 344862509 (329M) [application/x-gzip]
Saving to: ‘102flowers.tgz’

102flowers.tgz      100%[===================>] 328.89M  84.5MB/s    in 3.9s    

2020-06-05 17:45:11 (83.3 MB/s) - ‘102flowers.tgz’ saved [344862509/344862509]

--2020-06-05 17:45:22--  http://www.robots.ox.ac.uk/~vgg/data/flowers/102/setid.mat
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14989 (15K)
Saving to: ‘setid.mat’

setid.mat           100%[===================>]  14.64K  --.-KB/s    in 0.01s   

2020-06-05 17:45:22 (1.18 MB/s) - ‘setid.mat’ saved [14989/14989]


In [3]:
cd ~/data/flowers/

/root/data/flowers


In [4]:
import os
import numpy as np
import scipy.io
import keras
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
import cv2
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [5]:
!ls ~/data/flowers

imagelabels.mat  jpg  setid.mat


In [0]:
img_labels = scipy.io.loadmat("imagelabels.mat")
img_labels = img_labels["labels"]
img_labels = img_labels[0]
for i in range(len(img_labels)):
  img_labels[i] = img_labels[i] - 1

In [7]:
print("Number of labels:",len(img_labels))

Number of labels: 8189


In [0]:
train_x = []
train_y = []
dir = "jpg/"
for imgs in os.listdir(dir):
  img_num = int(imgs[7:11])-1
  train_y.append(img_labels[img_num])
  image = cv2.imread(os.path.join(dir, imgs))
  resized = cv2.resize(image, (150,150))
  normalized_img = cv2.normalize(resized, None, alpha=0, beta=1, 
                            norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
  train_x.append(normalized_img)
train_x = np.array(train_x)

In [9]:
print("Total number of images:",len(train_x))
print("Total number of labels:",len(train_y))

Total number of images: 8189
Total number of labels: 8189


In [0]:
#Dividing train,test,val data in 60%,20%,20% respectively

trainx, testx, trainy, testy = train_test_split(train_x, train_y, test_size=0.2, random_state=10)

trainx, valx, trainy, valy = train_test_split(trainx, trainy, test_size=0.2, random_state=10)

In [0]:
trainy = to_categorical(trainy)
testy = to_categorical(testy)
valy = to_categorical(valy)

In [12]:
print("Training data number:",len(trainx))
print("Testing data number:",len(testx))
print("Validation data number:",len(valx))

Training data number: 5240
Testing data number: 1638
Validation data number: 1311


In [13]:
print("Training labels number:",len(trainy))
print("Testing labels number:",len(testy))
print("Validation labels number:",len(valy))

Training labels number: 5240
Testing labels number: 1638
Validation labels number: 1311


In [0]:
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout

#Creating the neural net
model = Sequential()
#Layer 1
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.5))
#Layer 2
model.add(Conv2D(128, kernel_size=3, activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.5))
#Layer 3
model.add(Conv2D(128, kernel_size=3, activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.5))
#Layer 4
model.add(Conv2D(256, kernel_size=3, activation='relu'))
model.add(MaxPooling2D((2,2)))
#Input to Neural Network is flattened
model.add(Flatten())
#1st hidden layer with 512 neurons/nodes
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
#Output layer with 102 nodes for classifying 102 flowers
model.add(Dense(102, activation='softmax'))

In [0]:
#Compile the neural network
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [16]:
#Train the network
model.fit(trainx, trainy, validation_data = (valx, valy), epochs=50, batch_size=200)

Train on 5240 samples, validate on 1311 samples
Epoch 1/50
5240/5240 [==============================] - 13s 2ms/step - loss: 4.6304 - accuracy: 0.0242 - val_loss: 4.6011 - val_accuracy: 0.0320
Epoch 2/50
5240/5240 [==============================] - 5s 1ms/step - loss: 4.4608 - accuracy: 0.0406 - val_loss: 4.4140 - val_accuracy: 0.0572
Epoch 3/50
5240/5240 [==============================] - 5s 1ms/step - loss: 4.2319 - accuracy: 0.0552 - val_loss: 4.3523 - val_accuracy: 0.0580
Epoch 4/50
5240/5240 [==============================] - 5s 1ms/step - loss: 4.0755 - accuracy: 0.0672 - val_loss: 4.1791 - val_accuracy: 0.1007
Epoch 5/50
5240/5240 [==============================] - 5s 1ms/step - loss: 3.8783 - accuracy: 0.0887 - val_loss: 4.0306 - val_accuracy: 0.1175
Epoch 6/50
5240/5240 [==============================] - 5s 1ms/step - loss: 3.6881 - accuracy: 0.1130 - val_loss: 3.7938 - val_accuracy: 0.1388
Epoch 7/50
5240/5240 [==============================] - 5s 1ms/step - loss: 3.5072 - ac

In [0]:
#Save the model for further use
model.save("model.h5")

In [0]:
#Now the following code has also been implemented in the separate inference program file

#Load the model for testing
from keras.models import load_model
model = load_model("model.h5")

In [19]:
#Evaluate the model
score = model.evaluate(testx, testy) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

1638/1638 [==============================] - 1s 513us/step
Test loss: 2.4457970480633597
Test accuracy: 0.5030525326728821


In [20]:
#Predict output on sample input data
pred = model.predict(testx) 
pred = np.argmax(pred, axis = 1)[:10] 
label = np.argmax(testy,axis = 1)[:10] 

print("Predicted labels:",pred) 
print("Actual Labels:   ",label)

Predicted labels: [63 43 63 57  7 39 63 74 57 15]
Actual Labels:    [63 42 63 57  7 39 63 86 57 41]
